In [1]:
import requests
import pandas as pd
from mainnet_launch.constants import AUTO_ETH


def paginate_query(api_url: str, query: str, variables: dict, data_col: str) -> list[dict]:
    """
    Helper to page through a GraphQL connection using `first`/`skip`.

    :param api_url: GraphQL endpoint
    :param query: The GraphQL query string, expecting $first and $skip variables
    :param variables: Base variables (e.g. {"autoEthAddress": "..."}).
                      first/skip will be merged in each loop.
    :param batch_size: Number of items to fetch per request.
    :return: List of result dicts
    """

    all_records = []
    skip = 0

    while True:
        vars_with_pagination = {**variables, "first": 500, "skip": skip}
        resp = requests.post(api_url, json={"query": query, "variables": vars_with_pagination})
        resp.raise_for_status()
        batch = resp.json()["data"][data_col]

        if not batch:
            break

        all_records.extend(batch)
        skip += 500

    return all_records


def _get_subgraph_api(chain):
    if chain == "eth":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-eth-mainnet/api"
    elif chain == "base":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-base-mainnet/api"
    else:
        raise ValueError("bad chain", chain)

    return api_url


def fetch_autopool_rebalance_events_from_subgraph(autopool_eth_addr: str, chain: str) -> list[dict]:
    """
    Fetches all AutopoolRebalances entries for the given autopool.
    """
    subgraph_url = _get_subgraph_api(chain)

    query = """
    query($autoEthAddress: String!, $first: Int!, $skip: Int!) {
      autopoolRebalances(
        first: $first,
        skip: $skip,
        orderBy: id,
        orderDirection: desc,
        where: { autopool: $autoEthAddress }
      ) {
        transactionHash
        autopool
        timestamp

        tokenIn {
          id
          symbol
        }
        tokenInAmount
        tokenInValueInEth
        tokenInValueBaseAsset
        tokenInValueDenominatedIn
        destinationInAddress
        destinationInName

        tokenOut {
          id
          symbol
        }
        tokenOutAmount
        tokenOutValueInEth
        tokenOutValueBaseAsset
        tokenOutValueDenominatedIn
        destinationOutAddress
        destinationOutName
      }
    }
    """

    # Adjust `first` as needed; paginate_query will loop over skip increments of `first`
    return paginate_query(
        subgraph_url,
        query,
        variables={"autoEthAddress": autopool_eth_addr.lower(), "first": 1000, "skip": 0},
        data_col="autopoolRebalances",
    )

In [2]:
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_orm,
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
    natural_left_right_using_where,
    get_highest_value_in_field_where,
)
from mainnet_launch.database.schema.full import RebalanceEvents, RebalancePlans, Destinations
from web3 import Web3

rebalance_plans_df = get_full_table_as_df(
    RebalancePlans, where_clause=(RebalancePlans.autopool_vault_address == AUTO_ETH.autopool_eth_addr)
)
rebalance_plans_df["datetime_generated"] = pd.to_datetime(rebalance_plans_df["datetime_generated"], utc=True)

# 2. Fetch events and ensure UTC datetime
events = fetch_autopool_rebalance_events_from_subgraph(AUTO_ETH.autopool_eth_addr, "eth")
rebalance_event_df = pd.DataFrame.from_records(events)
rebalance_event_df["datetime_executed"] = pd.to_datetime(
    rebalance_event_df["timestamp"].astype(int), unit="s", utc=True
)

# 3. Normalize amount_out to float
rebalance_event_df["amount_out"] = rebalance_event_df["tokenOutAmount"].apply(lambda x: int(x) / 1e18)

2025-04-30 15:43:47,228 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-30 15:43:47,228 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-30 15:43:47,418 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-30 15:43:47,418 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-30 15:43:47,624 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-30 15:43:47,624 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-30 15:43:47,817 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-30 15:43:47,817 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [10]:
rebalance_plans_df["autopool_vault_address"]

0      0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
1      0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
2      0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
3      0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
4      0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
                          ...                    
625    0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
626    0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
627    0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
628    0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
629    0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
Name: autopool_vault_address, Length: 630, dtype: object

In [13]:
rebalance_event_df

,transactionHash,autopool,timestamp,tokenIn,tokenInAmount,tokenInValueInEth,tokenInValueBaseAsset,tokenInValueDenominatedIn,destinationInAddress,destinationInName,...,tokenOutValueInEth,tokenOutValueBaseAsset,tokenOutValueDenominatedIn,destinationOutAddress,destinationOutName,datetime_executed,amount_out,readable_token_in,readable_token_out,move_name
0,0xffe1690fa1ba7bf4b40f33f8ace5ee998eca02abe34d...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1736073071,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,77488231460393320448,77469785864928677898,77469785864928677898,77469785864928677898,0xba1462f43c6f60ebd1c62735c94e428ad073e01a,Tokemak-Wrapped Ether-pxETH/wETH,...,77526479464535523328,77526479464535523328,77526479464535523328,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2025-01-05 10:31:11+00:00,77.526479,pxethweth,WETH,WETH -> pxethweth
1,0xffb6a84d335844300ecdb6b6c306d5440071d8488ee0...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1727304503,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,185570447553138065408,185496862002029338711,185496862002029338711,185496862002029338711,0xe382bbd32c4e202185762ea433278f4ed9e6151e,Tokemak-Wrapped Ether-pxETH/wETH,...,185582339434752147456,185582339434752147456,185582339434752147456,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2024-09-25 22:48:23+00:00,185.582339,pxethweth,WETH,WETH -> pxethweth
2,0xfebf89276f199926e254a356dd21e36b6b315f6af587...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1736548271,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,183182318320755376128,183513559285856328261,183513559285856328261,183513559285856328261,0xc4eb861e7b66f593482a3d7e8adc314f6eeda30b,Tokemak-Wrapped Ether-Balancer pxETH/wETH Stab...,...,183659899400061550592,183659899400061550592,183659899400061550592,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2025-01-10 22:31:11+00:00,183.659899,pxETH/wETH,WETH,WETH -> pxETH/wETH
3,0xfe7d1b0ad530009a2e81a09accb1b0ee456b795f2169...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1727832755,{'id': '0xe080027bd47353b5d1639772b4a75e9ed365...,123894252946777800704,127346851971550743640,127346851971550743640,127346851971550743640,0x896ecc16ab4afff6ce0765a5b924baecd7fa455a,Tokemak-Wrapped Ether-osETH/rETH,...,127453903936895156224,127453903936895156224,127453903936895156224,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2024-10-02 01:32:35+00:00,127.453904,osETH-rETH,WETH,WETH -> osETH-rETH
4,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1726632143,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,314380623530208002048,316517522918850580284,316517522918850580284,316517522918850580284,0xe382bbd32c4e202185762ea433278f4ed9e6151e,Tokemak-Wrapped Ether-pxETH/wETH,...,314500371054658584576,314500371054658584576,314500371054658584576,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2024-09-18 04:02:23+00:00,314.500371,pxethweth,WETH,WETH -> pxethweth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,0x05172d31b8475ad5cc66f80ac1b57b360edea712f47c...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1745134295,{'id': '0x05ff47afada98a98982113758878f9a8b9fd...,19539138954586284032,20014705040916044670,20014705040916044670,20014705040916044670,0x40219bbda953ca811d2d0168dc806a96b84791d9,Tokemak-Wrapped Ether-Balancer weETH/rETH Stab...,...,20030346031616868352,20030346031616868352,20030346031616868352,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2025-04-20 07:31:35+00:00,20.030346,weETH/rETH,WETH,WETH -> weETH/rETH
397,0x04bfb10f0709842576f36040d12a4f2552d0ccf45d33...,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,1731753083,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,34859569148914380800,34916688822553696213,34916688822553696213,34916688822553696213,0xc4eb861e7b66f593482a3d7e8adc314f6eeda30b,Tokemak-Wrapped Ether-Balancer pxETH/wETH Stab...,...,34935243097201000448,34935243097201000448,34935243097201000448,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56,idle,2024-11-16 10:31:23+00:00,34.935243

In [3]:
rebalance_plans_df.columns

Index(['file_name', 'datetime_generated', 'autopool_vault_address', 'chain_id',
       'solver_address', 'rebalance_type', 'destination_out', 'token_out',
       'destination_in', 'token_in', 'move_name', 'amount_out',
       'amount_out_safe_value', 'min_amount_in', 'min_amount_in_safe_value',
       'out_spot_eth', 'out_dest_apr', 'in_spot_eth', 'in_dest_apr',
       'in_dest_adj_apr', 'apr_delta', 'swap_offset_period',
       'num_candidate_destinations', 'candidate_destinations_rank',
       'projected_swap_cost', 'projected_net_gain', 'projected_gross_gain',
       'projected_slippage'],
      dtype='object')

In [4]:
rebalance_plans_df[["datetime_generated", "amount_out", "destination_out"]].sort_values("datetime_generated")

,datetime_generated,amount_out,destination_out
0,2024-09-16 20:16:18+00:00,100.000000,0xE382BBd32C4E202185762eA433278f4ED9E6151E
1,2024-09-17 02:16:17+00:00,100.000000,0xE382BBd32C4E202185762eA433278f4ED9E6151E
2,2024-09-17 02:40:37+00:00,100.000000,0xE382BBd32C4E202185762eA433278f4ED9E6151E
3,2024-09-17 02:42:56+00:00,100.000000,0xE382BBd32C4E202185762eA433278f4ED9E6151E
4,2024-09-17 19:40:27+00:00,100.000000,0xE382BBd32C4E202185762eA433278f4ED9E6151E
...,...,...,...
625,2025-04-28 22:30:39+00:00,88.568189,0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9
626,2025-04-29 07:30:39+00:00,61.956502,0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9
627,2025-04-29 16:30:39+00:00,52.740886,0x40219bBda953ca811d2D0168Dc806a96b84791d9
628,2025-04-30 01:30:38+00:00,43.030803,0x3772973f8F399D74488D5cF3276C032E0afC8A6f


In [5]:
rebalance_event_df[["datetime_executed", "amount_out", "destinationOutAddress"]].sort_values("datetime_executed")

,datetime_executed,amount_out,destinationOutAddress
17,2024-09-17 19:42:23+00:00,100.000000,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
4,2024-09-18 04:02:23+00:00,314.500371,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
144,2024-09-18 16:22:35+00:00,146.034135,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
356,2024-09-19 02:57:47+00:00,134.705433,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
10,2024-09-19 13:26:23+00:00,335.831303,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
...,...,...,...
338,2025-04-29 07:31:47+00:00,61.956502,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
306,2025-04-29 16:31:47+00:00,52.740886,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
155,2025-04-30 01:31:47+00:00,43.030803,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
107,2025-04-30 10:31:47+00:00,46.165115,0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56


In [6]:
rebalance_plans_df["long_id"] = (
    rebalance_plans_df["amount_out"].astype(str)
    + " :: "
    + rebalance_plans_df["move_name"]
    + " :: "
    + rebalance_plans_df["datetime_generated"].astype(str)
)
rebalance_plans_df["long_id"].values[0]

'100.0 :: autoETH -> pxethweth :: 2024-09-16 20:16:18+00:00'

In [7]:
rebalance_event_df["readable_token_in"] = rebalance_event_df["tokenIn"].apply(lambda x: x["symbol"])
rebalance_event_df["readable_token_out"] = rebalance_event_df["tokenOut"].apply(lambda x: x["symbol"])
rebalance_event_df["move_name"] = (
    rebalance_event_df["readable_token_out"] + " -> " + rebalance_event_df["readable_token_in"]
)

rebalance_event_df["move_name"]

0       WETH -> pxethweth
1       WETH -> pxethweth
2      WETH -> pxETH/wETH
3      WETH -> osETH-rETH
4       WETH -> pxethweth
              ...        
396    WETH -> weETH/rETH
397    WETH -> pxETH/wETH
398       WETH -> pxsteth
399     WETH -> pxethweth
400       WETH -> pxsteth
Name: move_name, Length: 401, dtype: object

In [8]:
rebalance_event_df.destinationOutAddress

0      0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
1      0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
2      0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
3      0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
4      0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
                          ...                    
396    0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
397    0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
398    0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
399    0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
400    0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56
Name: destinationOutAddress, Length: 401, dtype: object

In [9]:
reba

NameError: name 'reba' is not defined

In [ ]:
rebalance_cols = rebalance_event_df.columns
rebalance_cols

In [ ]:
rebalance_event_df

In [ ]:
rebalance_event_df.columns

In [ ]:
plan_windows = {}

num_failed_to_find = 0
for _, evt in rebalance_event_df.iterrows():
    # use the event's transaction hash (adjust field name if different)
    tx_hash = evt.get("transactionHash")
    end_time = evt["datetime_executed"]
    start_time = end_time - pd.Timedelta(minutes=60)

    # filter plans in the 10-minute window before execution
    mask = (rebalance_plans_df["datetime_generated"] >= start_time) & (
        rebalance_plans_df["datetime_generated"] <= end_time
    )
    df_window = rebalance_plans_df.loc[mask].copy()
    plan_windows[f"rebalance_{tx_hash}"] = df_window

    if len(df_window == 0):
        num_failed_to_find += 1
        print(num_failed_to_find)


plan_windows["rebalance_0xffe1690fa1ba7bf4b40f33f8ace5ee998eca02abe34ded50637ea59d297210bb"]

In [ ]:
rebalance_event_df[
    rebalance_event_df["transactionHash"] == "0x031f16a929516757e642d58623ab75fbc458a14ed56a7218b0c9358abe962520"
].T

In [ ]:
pd.to_timedelta(
    (rebalance_event_df["datetime_executed"].min() - rebalance_plans_df["datetime_generated"].min())
).total_seconds() / 60

In [ ]:
sub_rebalance_df.dtypes

In [ ]:
rebalance_plans_df.dtypes

In [ ]:
rebalance_plans_df["datetime_generated"] - sub_rebalance_df["datetime_executed"].values[0]

In [ ]:
# def _infer_rebalance_plan_from_solver(all_plans:list[RebalancePlans], subgraph_rebalance_event:dict) -> RebalancePlans:
#     # cpu limited
#     target = (
#         subgraph_rebalance_event["autopool"].lower(),
#         subgraph_rebalance_event["destinationInAddress"].lower(),
#         subgraph_rebalance_event["destinationOutAddress"].lower(),
#     )

#     for plan in all_plans:
#         try:
#             candidate = (
#                 plan.autopool_vault_address.lower(),
#                 plan.destination_in.lower(),
#                 plan.destination_out.lower(),
#             )
#             if candidate == target:
#                 return plan
#         except AttributeError as e:
#             pass


# a = [_infer_rebalance_plan_from_solver(this_autopool_rebalance_plans, subgraph_event) for subgraph_event in rebalance_record_from_subgraph[:3]]
# a

In [ ]:
rebalance_event_df = pd.DataFrame.from_records(rebalance_record_from_subgraph)
rebalance_plan_df = get_full_table_as_df(
    RebalancePlans, where_clause=RebalancePlans.autopool_vault_address == ALL_AUTOPOOLS[0].autopool_eth_addr
)
rebalance_event_df["normalized_amount_out"] = rebalance_event_df["tokenOutAmount"].apply(lambda x: int(x) / 1e18)
rebalance_event_df

In [ ]:
rebalance_plan_df["destination_out"].value_counts()

In [ ]:
rebalance_plan_df

In [ ]:
rebalance_plan_df[["amount_out"]]

In [ ]:
rebalance_event_df = rebalance_event_df.sort_values("timestamp")
rebalance_event_df["normalized_amount_out"] = rebalance_event_df["tokenOutAmount"].apply(lambda x: int(x) / 1e18)
rebalance_event_df

In [ ]:
rebalance_event_df.columns

In [ ]:
rebalance_plan_df.columns

In [ ]:
matched = pd.merge_asof(
    rebalance_event_df[
        [
            "tokenOutAmount",
            "destinationInAddress",
            "destinationOutAddress",
            "datetime_executed",
            "normalized_amount_out",
        ]
    ],
    rebalance_plan_df[["file_name", "destination_in", "destination_out", "datetime_generated", "amount_out"]],
    left_on="datetime_executed",
    right_on="datetime_generated",
    left_by=[
        "normalized_amount_out",
    ],
    right_by=["amount_out"],
    direction="backward",  # pick the most recent plan at or before the event
    suffixes=("_evt", "_plan"),
)


matched

In [ ]:
matched[["destinationInAddress", "destination_in"]].value_counts()

In [ ]:
matched["execution_latency"] = pd.to_timedelta(matched["datetime_generated"] - matched["datetime_executed"])
matched

In [ ]:
plan: RebalancePlans = this_autopool_rebalance_plans[0]
plan.to_record()

In [ ]:
(plan.autopool_vault_address, plan.destination_in, plan.destination_out, plan.amount_out, plan.datetime_generated)

In [ ]:
# {'id': '0xffe1690fa1ba7bf4b40f33f8ace5ee998eca02abe34ded50637ea59d297210bb13000000',
#  'autopool': '0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56',
#  'timestamp': '1736073071',
#  'tokenIn': {'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588179d',
#   'symbol': 'pxethweth'},
#  'tokenInAmount': '77488231460393320448',
#  'tokenInValueInEth': '77469785864928677898',
#  'tokenInValueBaseAsset': '77469785864928677898',
#  'tokenInValueDenominatedIn': '77469785864928677898',
#  'destinationInAddress': '0xba1462f43c6f60ebd1c62735c94e428ad073e01a',
#  'destinationInName': 'Tokemak-Wrapped Ether-pxETH/wETH',
#  'tokenOut': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
#   'symbol': 'WETH'},
#  'tokenOutAmount': '77526479464535523328',
#  'tokenOutValueInEth': '77526479464535523328',
#  'tokenOutValueBaseAsset': '77526479464535523328',
#  'tokenOutValueDenominatedIn': '77526479464535523328',
#  'destinationOutAddress': '0x0a2b94f6871c1d7a32fe58e1ab5e6dea2f114e56',
#  'destinationOutName': 'idle'}

In [ ]:
rebalance_event_df = pd.DataFrame.from_records(rebalance_record_from_subgraph)
rebalance_event_df

In [ ]:
this_autopool_rebalance_plans[0].destination_in,

In [ ]:
this_autopool_rebalance_plans[0]

In [ ]:
def _rebalance_record_from_subgraph_to_rebalance_event_row(record: dict) -> RebalanceEvents:
    return RebalanceEvents(tx_hash=record["id"], autopool_vault_address=["autopool"])
    pass

In [ ]:
# class RebalanceEvents(Base):
#     __tablename__ = "rebalance_events"
#     # autopool, solver, time + (expiration (10 minutes)), token out, amount out,
#     tx_hash: Mapped[str] = mapped_column(ForeignKey("transactions.tx_hash"), primary_key=True)
#     # this has a pointer to a block
#     autopool_vault_address: Mapped[str]
#     rebalance_file_path: Mapped[str] = mapped_column(ForeignKey("rebalance_plans.file_name"))

#     quanity_out: Mapped[float] = mapped_column(nullable=False)
#     safe_value_out: Mapped[float] = mapped_column(nullable=False)
#     spot_value_out: Mapped[float] = mapped_column(nullable=False)
#     backing_value_out: Mapped[float] = mapped_column(nullable=False)  # not used but can be useful later

#     quanity_in: Mapped[float] = mapped_column(nullable=False)
#     safe_value_in: Mapped[float] = mapped_column(nullable=False)
#     spot_value_in: Mapped[float] = mapped_column(nullable=False)
#     backing_value_in: Mapped[float] = mapped_column(nullable=False)  # not used but can be useful later

#     actual_swap_cost: Mapped[float] = mapped_column(nullable=False)
#     break_even_days: Mapped[float] = mapped_column(nullable=False)
#     actual_slippage: Mapped[float] = mapped_column(nullable=False)

#     predicted_gain_during_swap_cost_off_set_period: Mapped[float] = mapped_column(nullable=False)
#     predicted_increase_after_swap_cost: Mapped[float] = mapped_column(nullable=False)

#     # consider adding in safe and acutal total supply here?
#     # only if wanted.

In [ ]:
possible

In [ ]:
rebalance_record_from_subgraph[0]